<a href="https://colab.research.google.com/github/vishalraut88/BERT_Models/blob/main/Word_embedding_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gensim
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
import pandas as pd
import numpy as np
import re
import string
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df_data = pd.read_csv("bbc-text.csv")

In [ ]:
api.info()["models"].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [ ]:
wv=api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
def sent2vec(sentence):
  vect_size=wv.vector_size
  wv_res=np.zeros(vect_size)
  count=1
  for word in sentence:
    if word in wv:
      wv_res+=wv[word]
      count+=1

  wv_res=wv_res/count
  return wv_res


In [ ]:
def text_cleaning(text):
    
    """
    Description : This function performs text cleaning that includes following:
        - lower the case
        - remove tags
        - remove special characters
        - remove punctuation
        - remove stopwords
    input parameters : Text that needs to be cleaned.
    returns : cleaned text after pre processing
    """
    text = text.lower()
    text = re.sub(r'http\S+', '', text)        
    text = re.sub("</?.*?>"," <> ",text)
    text = re.sub("(\\d|\\W)+"," ",text)        
    text = text.replace("_","")
    #remove punctuation
    text = [char for char in text if char not in string.punctuation]
    text = ''.join(text)
    stopwords = nltk.corpus.stopwords.words('english')
    stopwords.extend(['amp','like','û'])
    text = ' '.join([word for word in text.split() if word not in (stopwords)])            
    return text

In [ ]:
df_data.text = df_data.text.apply(text_cleaning)

In [ ]:
df_data["tokens"]=df_data.text.apply(lambda x:x.split())

In [ ]:
df_data["vectors"]=df_data["tokens"].apply(sent2vec)

In [ ]:
X = df_data['vectors'].to_list()
y = df_data['category'].to_list()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [ ]:

classifier.fit(X_train,y_train)

LogisticRegression()

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
predicted = classifier.predict(X_test)
print(classification_report(y_test, predicted))
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

               precision    recall  f1-score   support

     business       0.91      0.93      0.92       102
entertainment       1.00      0.95      0.97        77
     politics       0.92      0.90      0.91        84
        sport       1.00      0.99      1.00       102
         tech       0.93      0.97      0.95        80

     accuracy                           0.95       445
    macro avg       0.95      0.95      0.95       445
 weighted avg       0.95      0.95      0.95       445

Logistic Regression Accuracy: 0.950561797752809
